Scrub

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col,array,when
from pyspark.sql.types import ArrayType, IntegerType, ShortType

In [2]:
spark = SparkSession.builder\
    .appName('ETL')\
        .config("spark.driver.memory", "4g").getOrCreate()
spark

25/04/30 14:47:48 WARN Utils: Your hostname, prabina-bhushal resolves to a loopback address: 127.0.1.1; using 192.168.1.191 instead (on interface wlx54ef33f6c819)
25/04/30 14:47:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/30 14:47:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
network = spark.read.option("multiline","true").json("IgnoreFolder/df1.json")
provider = spark.read.option("multiline","true").json("IgnoreFolder/df2.json")

In [4]:
type(network)
type(provider)

network.printSchema()
provider.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- billing_code_type_version: string (nullable = true)
 |-- description: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negotiated_rates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- negotiated_prices: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- additional_information: string (nullable = true)
 |    |    |    |    |-- billing_class: string (nullable = true)
 |    |    |    |    |-- billing_code_modifier: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |-- expiration_date: string (nullable = true)
 |    |    |    |    |-- negotiated_rate: double (nullable = true)
 |    |    |    |    |-- negotiated_type: string (nullable = true)
 |    |    |    |    |-- service_code: array (nullable = true)
 |    |    |

In [5]:
network_exploded = network.withColumn("rates", explode("negotiated_rates"))
id_exploded = network_exploded.withColumn("id", explode("rates.provider_references"))
network_rates = id_exploded.withColumn("prices",explode("rates.negotiated_prices"))

In [6]:
in_network= network_rates.select(
    "billing_code",
    "billing_code_type",
    "negotiation_arrangement",
    col("id").alias("provider_group_id"),
    col("prices.negotiated_type").alias("negotiated_type"),
    col("prices.negotiated_rate").alias("negotiated_rate"),
    col("prices.billing_class").alias("billing_class"),
    col("prices.billing_code_modifier").alias("billing_code_modifier"),
    col("prices.service_code").alias("service_code")
)

In [7]:
in_network.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- provider_group_id: long (nullable = true)
 |-- negotiated_type: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- billing_class: string (nullable = true)
 |-- billing_code_modifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- service_code: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [8]:
in_network.show()

+------------+-----------------+-----------------------+-----------------+---------------+---------------+-------------+---------------------+------------+
|billing_code|billing_code_type|negotiation_arrangement|provider_group_id|negotiated_type|negotiated_rate|billing_class|billing_code_modifier|service_code|
+------------+-----------------+-----------------------+-----------------+---------------+---------------+-------------+---------------------+------------+
|       PEMG1|              CPT|                    ffs|         10001001|     negotiated|           70.0| professional|                 NULL|        [11]|
|       PEMG1|              CPT|                    ffs|         10002001|     negotiated|           70.0| professional|                 NULL|        [11]|
|       PEMG1|              CPT|                    ffs|         10003001|     negotiated|           70.0| professional|                 NULL|        [11]|
|       PEMG1|              CPT|                    ffs|        

In [9]:
provider_exploded = provider.withColumn("provider", explode("provider_groups"))
provider_npi = provider_exploded.withColumn("npi", explode("provider.npi"))

in_provider = provider_npi.select(
    col("provider_group_id"),
    col("npi").alias("npi"),
    col("provider.tin.type").alias("tin_type"),
    col("provider.tin.value").alias("tin")
)

in_provider.show(truncate=False)

+-----------------+----------+--------+----------+
|provider_group_id|npi       |tin_type|tin       |
+-----------------+----------+--------+----------+
|10001001         |1235233008|ein     |04-3267217|
|10001001         |1316041189|ein     |04-3267217|
|10001001         |1780788554|ein     |04-3267217|
|10001001         |1891068409|ein     |04-3267217|
|10001001         |1366459570|ein     |11-1562701|
|10001001         |1417915653|ein     |11-3358535|
|10001001         |1417915653|ein     |13-3888838|
|10002001         |1609829761|ein     |00-0004110|
|10002001         |1821482241|ein     |00-0004110|
|10002001         |1760986277|ein     |00-6980743|
|10002001         |1215075882|ein     |01-0550744|
|10002001         |1013917665|ein     |01-0555304|
|10002001         |1679780811|ein     |01-0555483|
|10002001         |1700093952|ein     |01-0555483|
|10002001         |1780072447|ein     |01-0555483|
|10002001         |1952532970|ein     |01-0555483|
|10002001         |1376647511|e

In [10]:
provider_final=in_provider.withColumn("tin_type",when(col("tin_type")=="ein",1)
                                      .when(col("tin_type")== "npi",2))

In [11]:
provider_final.show()

+-----------------+----------+--------+----------+
|provider_group_id|       npi|tin_type|       tin|
+-----------------+----------+--------+----------+
|         10001001|1235233008|       1|04-3267217|
|         10001001|1316041189|       1|04-3267217|
|         10001001|1780788554|       1|04-3267217|
|         10001001|1891068409|       1|04-3267217|
|         10001001|1366459570|       1|11-1562701|
|         10001001|1417915653|       1|11-3358535|
|         10001001|1417915653|       1|13-3888838|
|         10002001|1609829761|       1|00-0004110|
|         10002001|1821482241|       1|00-0004110|
|         10002001|1760986277|       1|00-6980743|
|         10002001|1215075882|       1|01-0550744|
|         10002001|1013917665|       1|01-0555304|
|         10002001|1679780811|       1|01-0555483|
|         10002001|1700093952|       1|01-0555483|
|         10002001|1780072447|       1|01-0555483|
|         10002001|1952532970|       1|01-0555483|
|         10002001|1376647511| 

Remove null value

In [12]:
from pyspark.sql.functions import expr

In [13]:
new_network=in_network.filter(in_network.billing_code.isNotNull())

In [14]:
# hash_network=remove_network.withColumn('service_code',hash('service_code'))

In [15]:
in_provider_hyphen = provider_final.withColumn("tin",expr("REPLACE(tin,'-','')"))

In [16]:
new_network.filter(in_network.billing_code_modifier.isNotNull()).count()

454512

In [17]:
# in_network_renamed=hash_network.withColumnRenamed('billing_class','bcIs')\
#             .withColumnRenamed('billing_code','bC')\
#             .withColumnRenamed('billing_code_type','bCT')\
#             .withColumnRenamed('negotiated_rate','negR')\
#             .withColumnRenamed('negotiated_type','negT')\
#             .withColumnRenamed('negotiation_arrangement','negA')\
#             .withColumnRenamed('service_code','poSH')
# in_network_renamed.show()

In [18]:
new_network.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- provider_group_id: long (nullable = true)
 |-- negotiated_type: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- billing_class: string (nullable = true)
 |-- billing_code_modifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- service_code: array (nullable = true)
 |    |-- element: string (containsNull = true)



Cast to array of integer

In [19]:
provider_cast = in_provider_hyphen.withColumn("tin_type",col("tin_type").cast(ShortType()))

In [20]:
provider_cast.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- npi: long (nullable = true)
 |-- tin_type: short (nullable = true)
 |-- tin: string (nullable = true)



In [21]:
in_network_cast = new_network.withColumn("service_code",col("service_code").cast(ArrayType(IntegerType()))
)

In [22]:
in_network_cast.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- provider_group_id: long (nullable = true)
 |-- negotiated_type: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- billing_class: string (nullable = true)
 |-- billing_code_modifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- service_code: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [23]:
print(in_network_cast)

DataFrame[billing_code: string, billing_code_type: string, negotiation_arrangement: string, provider_group_id: bigint, negotiated_type: string, negotiated_rate: double, billing_class: string, billing_code_modifier: array<string>, service_code: array<int>]


In [24]:
type(in_network_cast)
type(provider_cast)

pyspark.sql.dataframe.DataFrame

In [27]:
in_network_cast.write.parquet('IgnoreFolder/NetworkScrub.parquet')
provider_cast.write.parquet('IgnoreFolder/ProviderScrub.parquet')